# 1. Install required packages

In [13]:
%%capture 
!pip install https://releases.eratos.com/sdk/python/eratos-python-latest.zip
!pip install geopandas
!pip install keplergl
print('The Eratos SDK is installed')


# 2. Import required packages

In [14]:
from eratos.creds import AccessTokenCreds
from eratos.adapter import Adapter
import geopandas as gpd
from keplergl import KeplerGl
import os
import numpy as np
import pandas as pd
import time
import statistics
from shapely.geometry import Polygon, box
from shapely.strtree import STRtree
import fiona
import warnings
import unicodedata
import re
import json
from google.colab import files
import io







# 3. Update Client Folder Name

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Client_Folder_Name = 'Bridgewood'



ecreds = AccessTokenCreds(
  'your key',
  'your secret'
)


Mounted at /content/gdrive


# 4. Run Product Code

In [ ]:
# Welcome to the Eratos Forestry Product
#
# Copyright (C) Eratos Group Pty Ltd and its affiliates.
#
# Please enter your Client Folder name, identical to the folder you created
# and added the Client Property Geometry Files to, it is recommended to copy and paste it
# in between the quotation marks below, to the right of the Client_Folder_Name
# 



eadapter = Adapter(ecreds)


warnings.filterwarnings("ignore")


def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

print(slugify(Client_Folder_Name))

slug_client_name = slugify(Client_Folder_Name)

path =  '/content/gdrive/MyDrive/Eratos_Forestry_Product/Client_Files' + "/" + Client_Folder_Name


#Maximum Temperature of Warmest Month
burn_data = eadapter.Resource(ern='ern:e-pn.io:resource:eratos.blocks.modis.aumcd64a1')

# Access the gridded dataset via the Gridded API:
ds = burn_data.data().gapi()

var = 'burndate'

startTime = time.time()
# list to store files
geometry_files = []
df_regions = pd.DataFrame()
# Iterate directory
for file in os.listdir(path):

    if file.endswith('.kml'):
        gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
        for layer in fiona.listlayers(path + "/" + file,):
            
            geo = gpd.read_file(path + "/" + file, layer=layer,driver='KML').to_crs("EPSG:4326")
            #geo = gpd.read_file(path + "\\" + file, driver='KML').to_crs("EPSG:4326")
            df_regions = df_regions.append(geo)
        
    if file.endswith('.shp'):
        geo = gpd.read_file(path + "/" + file).to_crs("EPSG:4326")
        df_regions = df_regions.append(geo)


poly_list = []

#print(len(df_regions['geometry']))

for geom in df_regions['geometry']:

    bounds = geom.bounds
    
    minLon,minLat = bounds[0],bounds[1]
    maxLon,maxLat = bounds[2],bounds[3]

    lats = ds.get_subset_as_array('lat')
    lons = ds.get_subset_as_array('lon')
    spacingLat, spacingLon = lats[1]-lats[0], lons[1]-lons[0]

    minLatIdx, maxLatIdx = np.argmin(np.abs(lats-minLat)), np.argmin(np.abs(lats-maxLat))+1
    minLonIdx, maxLonIdx = np.argmin(np.abs(lons-minLon)), np.argmin(np.abs(lons-maxLon))+1
    times = ds.get_subset_as_array('time')

    if minLatIdx == maxLatIdx:
        maxLatIdx += 1
    if minLonIdx == maxLonIdx:
        maxLonIdx += 1    

    subLon = lons[minLonIdx:maxLonIdx]
    subLat = lats[minLatIdx:maxLatIdx]
    #x y y, x y y

    for j in range(len(subLon)):
        for k in range(len(subLat)):
            lat = subLat[k]
            lon = subLon[j]
            
            bottomLeftLat, bottomLeftLon = lat-0.5*spacingLat, lon-0.5*spacingLon
            topRightLat, topRightLon = lat+0.5*spacingLat, lon+0.5*spacingLon
            topLeftLat, topLeftLon = lat+0.5*spacingLat, lon-0.5*spacingLon
            bottomRightLat, bottomRightLon = lat-0.5*spacingLat, lon+0.5*spacingLon
            poly = Polygon([[topLeftLon, topLeftLat],[topRightLon, topRightLat],
            [bottomRightLon, bottomRightLat],[bottomLeftLon, bottomLeftLat]])
            
            poly_list.append(poly)



new_poly_list = []
for poly in poly_list:
    if poly not in new_poly_list: 
        new_poly_list.append(poly)

regions_R_tree = STRtree(df_regions['geometry'].to_list())
final_poly_list = []

for poly in new_poly_list:
    for geom in df_regions['geometry']:
        if poly.intersects(geom):
            final_poly_list.append(poly)


final_property_gdf = gpd.GeoDataFrame(geometry=final_poly_list)   
#print(len(final_poly_list))
id = range(len(final_poly_list))
final_property_gdf['id'] = id


output_directory_path = r'/content/gdrive/MyDrive/Eratos_Forestry_Product/Outputs/'  + slug_client_name + "_Outputs"
if os.path.exists(output_directory_path):
    print('folder exists, writing file')
else:
     os.mkdir(output_directory_path)


standardised_props_name = output_directory_path + "\\" +  slug_client_name + '_' +   'standardised_properties.geojson'
final_property_gdf.to_file(standardised_props_name, driver='GeoJSON')

grid_rate_25km = gpd.read_file('/content/gdrive/MyDrive/Eratos_Forestry_Product/Data_Files/grid_rate_25km_data_Q.geojson')
id = range(len(grid_rate_25km['geometry']))
grid_rate_25km['cell_id'] = id 

grid_rate_50km = gpd.read_file('/content/gdrive/MyDrive/Eratos_Forestry_Product/Data_Files/grid_rate_50km_data_Q.geojson')
id = range(len(grid_rate_50km['geometry']))
grid_rate_50km['cell_id'] = id 

grid_rate_100km = gpd.read_file('/content/gdrive/MyDrive/Eratos_Forestry_Product/Data_Files/grid_rate_100km_data_Q.geojson')
id = range(len(grid_rate_100km['geometry']))
grid_rate_100km['cell_id'] = id 

grid_rate_250km = gpd.read_file('/content/gdrive/MyDrive/Eratos_Forestry_Product/Data_Files/grid_rate_250km_data_Q.geojson')
id = range(len(grid_rate_250km['geometry']))
grid_rate_250km['cell_id'] = id 

tree_25 = STRtree(grid_rate_25km['geometry'])
tree_50 = STRtree(grid_rate_50km['geometry'])
tree_100 = STRtree(grid_rate_100km['geometry'])
tree_250 = STRtree(grid_rate_250km['geometry'])

tree_final = STRtree(final_property_gdf['geometry'])

print(len(final_property_gdf['geometry']))
t_len = len(final_property_gdf['geometry'])

print(f'The script will take around {str(round((t_len*2.66)/60,0))} minutes' )

grid_rate_25km_geom_np = grid_rate_25km['geometry'].to_numpy(dtype=None, copy=False)
count_store_25km = np.zeros(len(grid_rate_25km['geometry']))

grid_rate_50km_geom_np = grid_rate_50km['geometry'].to_numpy(dtype=None, copy=False)
count_store_50km = np.zeros(len(grid_rate_50km['geometry']))

grid_rate_100km_geom_np = grid_rate_100km['geometry'].to_numpy(dtype=None, copy=False)
count_store_100km = np.zeros(len(grid_rate_100km['geometry']))

grid_rate_250km_geom_np = grid_rate_250km['geometry'].to_numpy(dtype=None, copy=False)
count_store_250km = np.zeros(len(grid_rate_250km['geometry']))

for Pindex, Prow in final_property_gdf.iterrows():
    geom = Prow['geometry']
    str_tree = tree_25.query(geom)
    ##25
    for cell in str_tree:
        result = np.where(grid_rate_25km_geom_np == cell)
        count_store_25km[result] += 1/len(str_tree)
    ##50    
    str_tree = tree_50.query(geom)
    
    for cell in str_tree:
        result = np.where(grid_rate_50km_geom_np == cell)
        count_store_50km[result] += 1/len(str_tree)
    ##100    
    str_tree = tree_100.query(geom)
    
    for cell in str_tree:
        result = np.where(grid_rate_100km_geom_np == cell)
        count_store_100km[result] += 1/len(str_tree)

    ##250    
    str_tree = tree_250.query(geom)
    
    for cell in str_tree:
        result = np.where(grid_rate_250km_geom_np == cell)
        count_store_250km[result] += 1/len(str_tree)

grid_rate_25km['portfolio_count'] = count_store_25km
grid_rate_50km['portfolio_count'] = count_store_50km
grid_rate_100km['portfolio_count'] = count_store_100km
grid_rate_250km['portfolio_count'] = count_store_250km

df_portfolio_risk_25 = grid_rate_25km.loc[grid_rate_25km['portfolio_count'] > 0]
df_portfolio_risk_50 = grid_rate_50km.loc[grid_rate_50km['portfolio_count'] > 0]
df_portfolio_risk_100 = grid_rate_100km.loc[grid_rate_100km['portfolio_count'] > 0]
df_portfolio_risk_250 = grid_rate_250km.loc[grid_rate_250km['portfolio_count'] > 0]

# risk_25_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_risk_25km.geojson'
# risk_50_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_risk_50km.geojson'
# risk_100_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_risk_100km.geojson'
# risk_250_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_risk_250km.geojson'


# df_portfolio_risk_25.to_file(risk_25_name, driver='GeoJSON')
# df_portfolio_risk_50.to_file(risk_50_name, driver='GeoJSON')
# df_portfolio_risk_100.to_file(risk_100_name, driver='GeoJSON')
# df_portfolio_risk_250.to_file(risk_250_name, driver='GeoJSON')


final_stats_data_list = [df_portfolio_risk_25,df_portfolio_risk_50,df_portfolio_risk_100,df_portfolio_risk_250]

name = ['25km','50km','100km','250km']
weight_area_mean_rate_final = []
max_max_rate_final = []
mean_rate_final = []


for data in final_stats_data_list:
    total_portfolio_cells = data['portfolio_count'].sum()
    proportion_list = [] 
    for val in data['portfolio_count']:
        proportion_list.append(val/total_portfolio_cells)
    data['Weighted Influence (Area)'] = proportion_list
    weight_area_mean_rate = [a * b for a, b in zip(proportion_list, data['mean_grid_burn_rate'])]

    weight_area_mean_rate_final.append(sum(weight_area_mean_rate))
    max_max_rate_final.append(max(data['max_grid_burn_rate']))
    mean_rate_final.append(statistics.mean(data['mean_grid_burn_rate']))
    

portfolio_rates_25_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_rates_25km.csv'
portfolio_rates_50_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_rates_50km.csv'
portfolio_rates_100_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_rates_100km.csv'
portfolio_rates_250_name = output_directory_path + "\\" +  slug_client_name + '_' +   'portfolio_rates_250km.csv'


# final_stats_data_list[0].to_csv(portfolio_rates_25_name)   
# final_stats_data_list[1].to_csv(portfolio_rates_50_name)   
# final_stats_data_list[2].to_csv(portfolio_rates_100_name)   
# final_stats_data_list[3].to_csv(portfolio_rates_250_name)


final_output_df = pd.DataFrame()

final_output_df['grid'] = name
final_output_df['weight_area_mean_rate_final'] = weight_area_mean_rate_final
final_output_df['mean_rate_final'] = mean_rate_final
final_output_df['max_max_rate_final'] = max_max_rate_final

final_portfolio_rates_name = output_directory_path + "\\" +  slug_client_name + '_' +   'final_portfolio_rates.csv'

final_output_df.to_csv(final_portfolio_rates_name)   


with pd.ExcelWriter(output_directory_path + "\\" +  slug_client_name + '_' + 'Full_portfolio_rates.xlsx') as writer:  
    final_output_df.to_excel(writer, sheet_name='Rates_Summary')
    final_stats_data_list[0].to_excel(writer, sheet_name='portfolio_rates_25km')
    final_stats_data_list[1].to_excel(writer, sheet_name='portfolio_rates_50km')
    final_stats_data_list[2].to_excel(writer, sheet_name='portfolio_rates_100km')
    final_stats_data_list[3].to_excel(writer, sheet_name='portfolio_rates_250km')


f = open('/content/gdrive/MyDrive/Eratos_Forestry_Product/Scripts & Supporting/kepler_map_Liberty_config.json')

config = json.load(f)

f.close()

kepler_map_Liberty = KeplerGl(height = 1200,config=config)
kepler_map_Liberty.add_data(data=df_regions,name="Original_Properties")
kepler_map_Liberty.add_data(data=final_property_gdf,name="Standardised_Properties")
kepler_map_Liberty.add_data(data=final_stats_data_list[0],name="25km_cells")
kepler_map_Liberty.add_data(data=final_stats_data_list[1],name="50km_cells")
kepler_map_Liberty.add_data(data=final_stats_data_list[2],name="100km_cells")
kepler_map_Liberty.add_data(data=final_stats_data_list[3],name="250km_cells")


final_kepler_name = output_directory_path + "\\" +  slug_client_name + '_' +   'final_kepler.html'

kepler_map_Liberty.save_to_html(file_name=final_kepler_name,config=config)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


